In [2]:
import csv
from datetime import date

In [3]:
useful_data = []
counties = []
zip_codes = {}
f_date = date(2020, 3, 17)

# date, county, population, total cases
with open('data/cases_april.csv') as cases_csv:
    data_dump = csv.reader(cases_csv, delimiter=',')
    for row in data_dump:
        # grab data that has "Cases of COVID-19 in Colorado by County" in col. f
        if(row[5] == "Cases of COVID-19 in Colorado by County"):
            # date, county, pop, number of total cases
            # date diff
            ymd = row[9].split("/", 2)
            l_date = date(int(ymd[2]), int(ymd[0]), int(ymd[1]))
                    
            useful_row = [(l_date - f_date).days, row[1], row[4], row[7]]
            if(row[1] != 'Unknown Or Pending County'):
                useful_data.append(useful_row)

# new cases per day
useful_data.sort(key=lambda x: int(x[0]))
useful_data.sort(key=lambda x: x[1])
    
for i in range(len(useful_data)):
    if i == 0:
        useful_data[i].append(0)
    if((i+1) < len(useful_data)):
        new_cases = int(useful_data[i+1][3]) - int(useful_data[i][3])
        if(useful_data[i+1][1] == useful_data[i][1]):
            useful_data[i+1].append(new_cases)
        else:
            useful_data[i+1].append(0)
            
# 7 day avg new cases
for i in range(len(useful_data)):
    if i<7:
        useful_data[i].append(0)
    if((i+7) < len(useful_data)):
        case_total = 0
        for j in range(8):
            case_total += useful_data[i+j][4]
            
        case_total = case_total / 7
        
        if(useful_data[i+7][1] == useful_data[i][1]):
            useful_data[i+7].append(case_total)
        else:
            useful_data[i+7].append(0)
            
# 7 day avg delta
for i in range(len(useful_data)):
    if i<7:
        useful_data[i].append(0)
    if((i+7) < len(useful_data)):
        avg_delta = (useful_data[i+7][4] - useful_data[i][4]) / 7
        
        if(useful_data[i+7][1] == useful_data[i][1]):
            useful_data[i+7].append(avg_delta)
        else:
            useful_data[i+7].append(0)
            
# area of county
with open('data/areasqmi.csv') as area_csv:
    data_dump = csv.reader(area_csv, delimiter=',')
    for row in data_dump:
        county = row[0]
        counties.append(county)
        area = row[1]
        for data_row in useful_data:
            if data_row[1] == county:
                data_row.append(area)
                
# zip codes
with open('data/areacodes.csv') as area_codes:
    data_dump = csv.reader(area_codes, delimiter=',')

    for row in data_dump:
        county = row[1]
        zip_code = row[0]
        if(len(zip_code)>5):
            zip_code = zip_code[3:]
        zip_codes[zip_code] = county

mips = {}

with open('data/mipreport.csv') as mip:
    data_dump = csv.reader(mip, delimiter=',')
    flag = 1
    for row in data_dump:
        if flag == 1:
            flag = 0
            continue
            
        county = row[6]
        if county in mips:
            mips[county] += 1
        else:
            mips[county] = 1
            
counties = counties[1:]
for county in counties:
    if county in mips:
        continue
    else:
        mips[county] = 0
        
print(mips)

for data_row in useful_data:
    if data_row[1] in mips:
        data_row.append(mips[data_row[1]])
        
for row in useful_data[:]:
    if row[1] == 'International':
        useful_data.remove(row)
            
useful_data.insert(0,  ['Days Since March 17', 'County', 'Population', 'Total Sum Cases', 'New Cases per Day', 'Average Cases Per 7 Days', 'Average Daily Change in Cases Per 7 Days', 'Area of County', 'Number of MIP Charges'])
            
with open('data/data_sanitized_april.csv', 'w', newline='') as cases_csv:
    data_dump = csv.writer(cases_csv, delimiter=',')
    data_dump.writerows(useful_data)

{'Weld': 33, 'El Paso': 11, 'Arapahoe': 11, 'Boulder': 43, 'Eagle': 10, 'Garfield': 10, 'Larimer': 2, 'Rio Grande': 1, 'Adams': 0, 'Alamosa': 0, 'Archuleta': 0, 'Baca': 0, 'Bent': 0, 'Broomfield': 0, 'Chaffee': 0, 'Cheyenne': 0, 'Clear Creek': 0, 'Conejos': 0, 'Costilla': 0, 'Crowley': 0, 'Custer': 0, 'Delta': 0, 'Denver': 0, 'Dolores': 0, 'Douglas': 0, 'Elbert': 0, 'Fremont': 0, 'Gilpin': 0, 'Grand': 0, 'Gunnison': 0, 'Hinsdale': 0, 'Huerfano': 0, 'Jackson': 0, 'Jefferson': 0, 'Kiowa': 0, 'Kit Carson': 0, 'La Plata': 0, 'Lake': 0, 'Las Animas': 0, 'Lincoln': 0, 'Logan': 0, 'Mesa': 0, 'Mineral': 0, 'Moffat': 0, 'Montezuma': 0, 'Montrose': 0, 'Morgan': 0, 'Otero': 0, 'Ouray': 0, 'Park': 0, 'Phillips': 0, 'Pitkin': 0, 'Prowers': 0, 'Pueblo': 0, 'Rio Blanco': 0, 'Routt': 0, 'Saguache': 0, 'San Juan': 0, 'San Miguel': 0, 'Sedgwick': 0, 'Summit': 0, 'Teller': 0, 'Washington': 0, 'Yuma': 0}
